In [25]:
import os
from datetime import datetime
import numpy as np
import pandas as pd

PATH = "games.csv" 
PROCESSED_DIR = "processed"
AGG_DIR = "aggregates"
DISC_DIR = "discoverability"
CURRENT_YEAR = datetime.now().year

for d in [PROCESSED_DIR, AGG_DIR, DISC_DIR]: #making directories for my csv files
    os.makedirs(d, exist_ok=True)

def load_and_clean():
    df = pd.read_csv(PATH)
    
    columns = [   #defining the columns I am going to need
        "AppID",
        "Name",
        "Release date",
        "Estimated owners",
        "Peak CCU",
        "Required age",
        "Price",
        "Genres",
        "Tags",
        "User score",
        "Positive",
        "Negative",
        "Average playtime forever",
        "Average playtime two weeks",
        "Median playtime forever",
        "Median playtime two weeks",
        "Developers",
        "Publishers",
        "Categories",
        "Recommendations",
    ]
    columns = [c for c in columns if c in df.columns] #keeping the columns from the file
    df = df[columns]

    if "Release date" in df.columns: #converting the release date columns' string entries to date format
        def parse_date(x):
            try:
                return pd.to_datetime(x, errors="coerce")
            except:    #return missing date if it fails
                return pd.NaT
        df["Release date"] = df["Release date"].apply(parse_date) #applying it to every row
        df["release_year"] = df["Release date"].dt.year  #extracting the year and saving it in a new column
    else:
        df["release_year"] = np.nan
    for col in [                                         #convert any numerical strings into actual number format
        "Price", "Positive", "Negative", "User score",
        "Average playtime forever", "Average playtime two weeks",
        "Median playtime forever", "Median playtime two weeks"
    ]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            
    df = df.dropna(subset=["Name"])        #making sure every game has a name and if not just dropping the row because its required

    for col in ["Price", "Positive", "Negative"]:
        if col in df.columns:
            df[col] = df[col].fillna(0)  #missing or invalid numbers will be 0 instead

    outpath = f"{PROCESSED_DIR}/steam_clean.csv"
    df.to_csv(outpath, index=False) #saving the clearned dataset
    return df 

In [26]:
#feature engineering

def engineer_features(df_clean: pd.DataFrame):
    df = df_clean.copy()

    if "Positive" in df.columns and "Negative" in df.columns:
        df["review_count"] = df["Positive"] + df["Negative"] #New total number of reviews column will be positive + negative
    else:
        raise ValueError("Columns 'Positive' and 'Negative' are required to compute review_count.")

    df["rating"] = df["Positive"] / df["review_count"].replace(0, np.nan) #New column called rating will positive / (positive + negative)
    df["log_review_count"] = np.log1p(df["review_count"]) #New column, computing log of the review count log1p(x) = log(1 + x)
    df["visibility_score"] = df["rating"] * df["log_review_count"] #New column, computing visibility (how popular and well reviewed the game is) = rating * log(1 + reviews)

    if "Tags" in df.columns: #new column, counting how many tags each game has
        df["tag_count"] = df["Tags"].fillna("").apply( 
            lambda x: len([t for t in str(x).split(",") if t.strip()])
        )
    else:
        df["tag_count"] = 0
    if "Genres" in df.columns: #new column, checking if a game's genre list includes indie
        df["is_indie"] = df["Genres"].fillna("").str.contains("Indie", case=False)
    else:
        df["is_indie"] = False

    if "Price" in df.columns:
        def bucket_price(p): #sorting the game into price categories
            if p <= 5:
                return "Very Cheap (<= $5)"
            elif p <= 15:
                return "Cheap ($5–15)"
            elif p <= 40:
                return "Mid ($15–40)"
            else:
                return "Expensive ($40+)"
        df["price_bucket"] = df["Price"].apply(bucket_price) #new column
    else:
        df["price_bucket"] = "Unknown"
    if "release_year" in df.columns: #new column, checking how old each game is
        df["age_years"] = CURRENT_YEAR - df["release_year"]
    else:
        df["age_years"] = np.nan

    outpath = f"{PROCESSED_DIR}/steam_features.csv"
    df.to_csv(outpath, index=False)
    return df

In [27]:
def build_aggregates(df_features: pd.DataFrame):
    df = df_features.copy()

    if "Genres" in df.columns: #grouping names by genre and then calculating the number of games, average rating, and average visibility of the group
        genre_stats = ( 
            df.groupby("Genres")
            .agg(
                n_games=("Name", "count"),
                avg_rating=("rating", "mean"),
                avg_visibility=("visibility_score", "mean")
            )
            .reset_index()
        )
        genre_stats.to_csv(f"{AGG_DIR}/genre_stats.csv", index=False) #saving the data to a new csv file
    else:
        genre_stats = None

    if "release_year" in df.columns: #grouping by year to get how many games were released and average rating per year
        year_stats = (
            df.groupby("release_year")
            .agg(
                n_games=("Name", "count"),
                avg_rating=("rating", "mean")
            )
            .reset_index()
        )
        year_stats.to_csv(f"{AGG_DIR}/year_stats.csv", index=False) #saving to new csv file
    else:
        year_stats = None

    if "Genres" in df.columns and "release_year" in df.columns:
        genre_year_stats = (
            df.groupby(["Genres", "release_year"]) #grouping my dataset by genre and release year
            .agg( #calculating how many games came out and average visibility score for each genre year pair.
                n_games=("Name", "count"),
                avg_visibility=("visibility_score", "mean")
            )  
            .reset_index() 
        )
        genre_year_stats.to_csv(f"{AGG_DIR}/genre_year_stats.csv", index=False) #saving the results in a new csv
    else:
        genre_year_stats = None
    return genre_stats, year_stats, genre_year_stats

In [28]:
def discoverability(df_features: pd.DataFrame):
    df = df_features.copy()

    if "Genres" in df.columns: #ranking visibility of games within each genre
        df["visibility_rank_in_genre"] = (
            df.groupby("Genres")["visibility_score"]
            .rank(ascending=False, method="dense")
        )
    else:
        df["visibility_rank_in_genre"] = np.nan
    #looking to see if there are any well-rated games with not that many reviews, but with at least 1
    high_rating = 0.9         
    low_reviews = 40           

    hidden_gems = df[
        (df["rating"] >= high_rating) &
        (df["review_count"] > 0) &       
        (df["review_count"] <= low_reviews)
    ].copy()

    hidden_gems.to_csv(f"{DISC_DIR}/hidden_gems.csv", index=False)
    top_visible = df.sort_values("visibility_score", ascending=False).head(200) #most visible games overall
    top_visible.to_csv(f"{DISC_DIR}/top_visible_games.csv", index=False)
    return hidden_gems, top_visible

In [ ]:
def main(): #running all my code
    df_clean = load_and_clean()           
    df_feat = engineer_features(df_clean) 
    build_aggregates(df_feat)             
    discoverability(df_feat)      
main()
